# Nurse staffing strategies for enhanced patient care

Analysis of the Center for Medicare & Medicaid Services Nurse Staffing
Dataset

Matthew Bain  
2024-03-22

I analyze a medical staffing dataset and identify avenues to improve
work satisfaction among nurses and the quality of care provided at
United States medical institutions.

\[…\]

## Imports

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy import stats
from great_tables import GT
from itertools import combinations
from pandas.plotting import scatter_matrix
from IPython.display import (
    display as display3,
    Markdown
)

from src.paths import get_path_to
from src.stylesheet import customize_plots
from src.inspection import make_df, display, display2

## The dataset

### Load the data

We begin by exploring the data to get to know the features and patterns
on which we will base our analysis.

In [4]:
if 'data' not in locals():
    data = pd.read_csv(
        get_path_to("data", "raw", "PBJ_Daily_Nurse_Staffing_Q1_2024.zip"),
        encoding='ISO-8859-1',
        low_memory=False
    )
else:
    print("data loaded.")

### Inspect the data

In [5]:
GT(data.sample(10))

In [7]:
df = data.describe().round(1)
GT(df.reset_index())

### Group the features

We note that there are 91 records per provider
(`len(data["WorkDate"].unique())`) and 1,330,966 records in the table
overall. The following table, which collapses the raw data across
providers, thus has 14,626 $\left( \frac{1330966}{91} \right)$ entries.

In [8]:
df = (
    data.loc[:, [
        "STATE",
        "COUNTY_NAME", "COUNTY_FIPS",
        "CITY",
        "PROVNAME", "PROVNUM",
    ]]
    .value_counts()
    .to_frame()
    .rename(columns={0: 'Counts'})
)
GT(df.reset_index().head(n=20))

In [10]:
GT(data[["CY_Qtr", "WorkDate", "MDScensus"]].head())

In [11]:
# Normalize feature names
# data.rename(columns=lambda name: name.replace('_', ' ').capitalize())

# Group by location for geographic view
# data.groupby(by=["STATE", "CITY"])["Hrs_NAtrn"].sum()

# Pivot by date for historical view

# Query
# (data
#  .query("Hrs_NAtrn > 0 and STATE in ['AL', 'AZ'] and WorkDate > 20240315")
# )

# Subset feature groups
data.filter(regex=r'^Hrs_[^_]+$', axis='columns')
# data.filter(like="Hrs_RNDON", axis="columns")

# Drop
# data.drop(columns=data.filter(regex="^CY_Qtr$").columns, inplace=False)

1330966 rows × 8 columns

### Clean the data

In [12]:
# Normalize feature names (lowercase and remove underscores)

# Normalize categorical features (make a column title case)
# data['CITY'].str.title()

## Explore the dataset

### Visualize distributions

### Visualize relationships

In [13]:
attributes = ["Hrs_RN", "Hrs_LPN_ctr", "Hrs_CNA", "Hrs_NAtrn", "Hrs_MedAide"]
n = len(attributes)

fig, axs = plt.subplots(n, n, figsize=(8, 8))
scatter_matrix(
    data[attributes].sample(200),
    ax=axs, alpha=.7,
    hist_kwds=dict(bins=15, linewidth=0)
)
fig.align_ylabels(axs[:, 0])
fig.align_xlabels(axs[-1, :])
for ax in axs.flatten():
    ax.tick_params(axis='both', which='both', length=3.5)

# save_fig("scatter_matrix_plot")

plt.show()

### Compare groups

> **Note 1: \[Recommendation\].**

In [14]:
##| fig-subcap:
##|   - Average working hours with 95% confidence intervals.
##|   - Results of group comparisons by independent t-tests.

N_GROUPS = 6
N_LEVELS = 1

data_ = np.random.normal(loc=5, scale=3.0, size=(N_GROUPS, N_LEVELS, 10))

# Calculate averages and confidence intervals
averages = np.mean(data_, axis=2)
conf_intervals = np.zeros_like(averages, dtype=float)

for group_idx in range(N_GROUPS):
    for level_idx in range(N_LEVELS):
        interval = stats.t.interval(
            0.95,
            len(data_[group_idx, level_idx]) - 1,
            loc=np.mean(data_[group_idx, level_idx]),
            scale=stats.sem(data_[group_idx, level_idx])
        )

        # Use upper bound
        conf_intervals[group_idx, level_idx] = np.abs(
            interval[1] - averages[group_idx, level_idx]
        )

# -- Plot grouped bars with confidence intervals -----------------------------

width = 0.2
colors = plt.cm.Blues_r(np.linspace(.15, .85, N_LEVELS))
line_thickness = 0.6
stagger_amount = 0.8

fig, ax = plt.subplots()

for level_idx in range(N_LEVELS):
    bars = ax.bar(
        np.arange(N_GROUPS) + level_idx * width - (width * (N_LEVELS - 1) / 2),
        averages[:, level_idx],
        yerr=conf_intervals[:, level_idx],
        width=width,
        edgecolor="white",
        alpha=0.85,
        # capsize=3,
        color=colors[level_idx],
        error_kw={'elinewidth': line_thickness, 'capsize': 0},
        label=f'Level {level_idx + 1}',
    )

# Style
ax.set_ylabel('Values')

group_labels = [f'Group {i}' for i in range(1, N_GROUPS + 1)]
ax.set_xticks(np.arange(N_GROUPS))
ax.set_xticklabels(group_labels, rotation=60, ha='right')

# ax.legend(title='', bbox_to_anchor=(1.05, 1), loc='upper left')

# -- Add staggered sigbars and asterisks for select btwn-group comparisons ---

significance_level = 0.09
stagger_index = 0
stats_list = []

for comb in combinations(range(N_GROUPS), 2):
    group1_center = ax.get_xticks()[comb[0]]
    group2_center = ax.get_xticks()[comb[1]]

    t_stat, p_value = stats.ttest_ind(
        data_[comb[0], :, :].flatten(),
        data_[comb[1], :, :].flatten()
    )

    if p_value < significance_level:
        tallest_bar_height = np.max(averages) + np.max(conf_intervals) + 0.5

        # Adjust the stagger amount
        significance_height = (
            tallest_bar_height
            + np.max(conf_intervals) * 0.07
            + stagger_index * stagger_amount
        )

        # Plot staggered lines aligned with the midpoints of compared groups
        ax.plot(
            [group1_center, group2_center],
            [significance_height] * 2,
            color='black',
            lw=line_thickness
        )

        # Plot asterisks aligned with the center of the significance bars
        asterisks = (
            '*' * sum([p_value < alpha for alpha in [0.01, 0.001, 0.0001]])
        )
        ax.text(
            (group1_center + group2_center) / 2,
            significance_height,
            asterisks,
            ha='center',
            va='bottom',
            fontsize=10
        )

        # Increment the index for staggered bars
        stagger_index += 1

        # Store significant comparisons, t values, and sample sizes
        sample_size1 = len(data_[comb[0], :, :].flatten())
        sample_size2 = len(data_[comb[1], :, :].flatten())
        stats_list.append({
            "Comparison":
                f'{group_labels[comb[0]]} vs {group_labels[comb[1]]}',
            "p-value":
                f"{p_value:.4f}",
            "t-statistic":
                f"{t_stat:.4f}",
            "Sample Size": (
                f'{group_labels[comb[0]]} = {sample_size1}, '
                f'{group_labels[comb[1]]} = {sample_size2}'
            )
        })

# Style and show
ax.spines[['top', 'right']].set_visible(False)
ax.spines[['bottom', 'left']].set_visible(False)
ax.set_axisbelow(True)

ax.grid(axis='y')
plt.tight_layout()
plt.show()

stats_df = pd.DataFrame(stats_list)

<table>
<colgroup>
<col style="width: 100%" />
</colgroup>
<tbody>
<tr class="odd">
<td
style="text-align: center;"><div id="tuuhhhuooa" style="padding-left:0px;padding-right:0px;padding-top:10px;padding-bottom:10px;overflow-x:auto;overflow-y:auto;width:auto;height:auto;">
<style>
#tuuhhhuooa table {
          font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, Oxygen, Ubuntu, Cantarell, 'Helvetica Neue', 'Fira Sans', 'Droid Sans', Arial, sans-serif;
          -webkit-font-smoothing: antialiased;
          -moz-osx-font-smoothing: grayscale;
        }
&#10;#tuuhhhuooa thead, tbody, tfoot, tr, td, th { border-style: none !important; }
 tr { background-color: transparent !important; }
#tuuhhhuooa p { margin: 0 !important; padding: 0 !important; }
 #tuuhhhuooa .gt_table { display: table !important; border-collapse: collapse !important; line-height: normal !important; margin-left: auto !important; margin-right: auto !important; color: #333333 !important; font-size: 16px !important; font-weight: normal !important; font-style: normal !important; background-color: #FFFFFF !important; width: auto !important; border-top-style: solid !important; border-top-width: 2px !important; border-top-color: #A8A8A8 !important; border-right-style: none !important; border-right-width: 2px !important; border-right-color: #D3D3D3 !important; border-bottom-style: solid !important; border-bottom-width: 2px !important; border-bottom-color: #A8A8A8 !important; border-left-style: none !important; border-left-width: 2px !important; border-left-color: #D3D3D3 !important; }
 #tuuhhhuooa .gt_caption { padding-top: 4px !important; padding-bottom: 4px !important; }
 #tuuhhhuooa .gt_title { color: #333333 !important; font-size: 125% !important; font-weight: initial !important; padding-top: 4px !important; padding-bottom: 4px !important; padding-left: 5px !important; padding-right: 5px !important; border-bottom-color: #FFFFFF !important; border-bottom-width: 0 !important; }
 #tuuhhhuooa .gt_subtitle { color: #333333 !important; font-size: 85% !important; font-weight: initial !important; padding-top: 3px !important; padding-bottom: 5px !important; padding-left: 5px !important; padding-right: 5px !important; border-top-color: #FFFFFF !important; border-top-width: 0 !important; }
 #tuuhhhuooa .gt_heading { background-color: #FFFFFF !important; text-align: center !important; border-bottom-color: #FFFFFF !important; border-left-style: none !important; border-left-width: 1px !important; border-left-color: #D3D3D3 !important; border-right-style: none !important; border-right-width: 1px !important; border-right-color: #D3D3D3 !important; }
 #tuuhhhuooa .gt_bottom_border { border-bottom-style: solid !important; border-bottom-width: 2px !important; border-bottom-color: #D3D3D3 !important; }
 #tuuhhhuooa .gt_col_headings { border-top-style: solid !important; border-top-width: 2px !important; border-top-color: #D3D3D3 !important; border-bottom-style: solid !important; border-bottom-width: 2px !important; border-bottom-color: #D3D3D3 !important; border-left-style: none !important; border-left-width: 1px !important; border-left-color: #D3D3D3 !important; border-right-style: none !important; border-right-width: 1px !important; border-right-color: #D3D3D3 !important; }
 #tuuhhhuooa .gt_col_heading { color: #333333 !important; background-color: #FFFFFF !important; font-size: 100% !important; font-weight: normal !important; text-transform: inherit !important; border-left-style: none !important; border-left-width: 1px !important; border-left-color: #D3D3D3 !important; border-right-style: none !important; border-right-width: 1px !important; border-right-color: #D3D3D3 !important; vertical-align: bottom !important; padding-top: 5px !important; padding-bottom: 5px !important; padding-left: 5px !important; padding-right: 5px !important; overflow-x: hidden !important; }
 #tuuhhhuooa .gt_column_spanner_outer { color: #333333 !important; background-color: #FFFFFF !important; font-size: 100% !important; font-weight: normal !important; text-transform: inherit !important; padding-top: 0 !important; padding-bottom: 0 !important; padding-left: 4px !important; padding-right: 4px !important; }
 #tuuhhhuooa .gt_column_spanner_outer:first-child { padding-left: 0 !important; }
 #tuuhhhuooa .gt_column_spanner_outer:last-child { padding-right: 0 !important; }
 #tuuhhhuooa .gt_column_spanner { border-bottom-style: solid !important; border-bottom-width: 2px !important; border-bottom-color: #D3D3D3 !important; vertical-align: bottom !important; padding-top: 5px !important; padding-bottom: 5px !important; overflow-x: hidden !important; display: inline-block !important; width: 100% !important; }
 #tuuhhhuooa .gt_spanner_row { border-bottom-style: hidden !important; }
 #tuuhhhuooa .gt_group_heading { padding-top: 8px !important; padding-bottom: 8px !important; padding-left: 5px !important; padding-right: 5px !important; color: #333333 !important; background-color: #FFFFFF !important; font-size: 100% !important; font-weight: initial !important; text-transform: inherit !important; border-top-style: solid !important; border-top-width: 2px !important; border-top-color: #D3D3D3 !important; border-bottom-style: solid !important; border-bottom-width: 2px !important; border-bottom-color: #D3D3D3 !important; border-left-style: none !important; border-left-width: 1px !important; border-left-color: #D3D3D3 !important; border-right-style: none !important; border-right-width: 1px !important; border-right-color: #D3D3D3 !important; vertical-align: middle !important; text-align: left !important; }
 #tuuhhhuooa .gt_empty_group_heading { padding: 0.5px !important; color: #333333 !important; background-color: #FFFFFF !important; font-size: 100% !important; font-weight: initial !important; border-top-style: solid !important; border-top-width: 2px !important; border-top-color: #D3D3D3 !important; border-bottom-style: solid !important; border-bottom-width: 2px !important; border-bottom-color: #D3D3D3 !important; vertical-align: middle !important; }
 #tuuhhhuooa .gt_from_md> :first-child { margin-top: 0 !important; }
 #tuuhhhuooa .gt_from_md> :last-child { margin-bottom: 0 !important; }
 #tuuhhhuooa .gt_row { padding-top: 8px !important; padding-bottom: 8px !important; padding-left: 5px !important; padding-right: 5px !important; margin: 10px !important; border-top-style: solid !important; border-top-width: 1px !important; border-top-color: #D3D3D3 !important; border-left-style: none !important; border-left-width: 1px !important; border-left-color: #D3D3D3 !important; border-right-style: none !important; border-right-width: 1px !important; border-right-color: #D3D3D3 !important; vertical-align: middle !important; overflow-x: hidden !important; }
 #tuuhhhuooa .gt_stub { color: #333333 !important; background-color: #FFFFFF !important; font-size: 100% !important; font-weight: initial !important; text-transform: inherit !important; border-right-style: solid !important; border-right-width: 2px !important; border-right-color: #D3D3D3 !important; padding-left: 5px !important; padding-right: 5px !important; }
 #tuuhhhuooa .gt_stub_row_group { color: #333333 !important; background-color: #FFFFFF !important; font-size: 100% !important; font-weight: initial !important; text-transform: inherit !important; border-right-style: solid !important; border-right-width: 2px !important; border-right-color: #D3D3D3 !important; padding-left: 5px !important; padding-right: 5px !important; vertical-align: top !important; }
 #tuuhhhuooa .gt_row_group_first td { border-top-width: 2px !important; }
 #tuuhhhuooa .gt_row_group_first th { border-top-width: 2px !important; }
 #tuuhhhuooa .gt_striped { background-color: rgba(128,128,128,0.05) !important; }
 #tuuhhhuooa .gt_table_body { border-top-style: solid !important; border-top-width: 2px !important; border-top-color: #D3D3D3 !important; border-bottom-style: solid !important; border-bottom-width: 2px !important; border-bottom-color: #D3D3D3 !important; }
 #tuuhhhuooa .gt_sourcenotes { color: #333333 !important; background-color: #FFFFFF !important; border-bottom-style: none !important; border-bottom-width: 2px !important; border-bottom-color: #D3D3D3 !important; border-left-style: none !important; border-left-width: 2px !important; border-left-color: #D3D3D3 !important; border-right-style: none !important; border-right-width: 2px !important; border-right-color: #D3D3D3 !important; }
 #tuuhhhuooa .gt_sourcenote { font-size: 90% !important; padding-top: 4px !important; padding-bottom: 4px !important; padding-left: 5px !important; padding-right: 5px !important; text-align: left !important; }
 #tuuhhhuooa .gt_left { text-align: left !important; }
 #tuuhhhuooa .gt_center { text-align: center !important; }
 #tuuhhhuooa .gt_right { text-align: right !important; font-variant-numeric: tabular-nums !important; }
 #tuuhhhuooa .gt_font_normal { font-weight: normal !important; }
 #tuuhhhuooa .gt_font_bold { font-weight: bold !important; }
 #tuuhhhuooa .gt_font_italic { font-style: italic !important; }
 #tuuhhhuooa .gt_super { font-size: 65% !important; }
 #tuuhhhuooa .gt_footnote_marks { font-size: 75% !important; vertical-align: 0.4em !important; position: initial !important; }
 #tuuhhhuooa .gt_asterisk { font-size: 100% !important; vertical-align: 0 !important; }
 &#10;</style>
&#10;<table class="gt_table do-not-create-environment cell"
data-quarto-postprocess="true" data-quarto-disable-processing="false"
data-quarto-bootstrap="false">
<tbody class="gt_table_body">
</tbody>
</table>
&#10;
</div>
        </td>
</tr>
</tbody>
</table>

Table 2: Results of group comparisons by independent t-tests.

## Feature engineer

### Join geographical data

In [16]:
if 'uscities' not in locals():
    uscities = pd.read_csv(
        get_path_to("data", "raw", "uscities", "uscities.csv"),
        encoding='ISO-8859-1',
        low_memory=False
    )
else:
    print("data loaded.")
    
GT(uscities.sample(5))

In [17]:
data_ = data.copy()
data_["CITY"] = data_['CITY'].str.title()
data_.rename(columns={"CITY": "city"}, inplace=True)

In [18]:
# Join
data_geo = data_.join(uscities.set_index("city")[["lat", "lng"]], on="city")
GT(data_geo.head())

In [19]:
# Sum aggregate hours columns
data_geo["total hours"] = (
    data_geo
    .filter(regex=r'^Hrs_[^_]+$', axis='columns')
    .sum(axis=1)
)

# Group hours by city (collapse across date)
data_geo_ = (
    data_geo
    .dropna()
    .groupby(by=["city"], as_index=False)
    .agg({"total hours": "sum", "lat": "first", "lng": "first"})
    .rename(columns={"total hours": "total_hours_sum"})
)

data_geo_

4646 rows × 4 columns

In [34]:
import geopandas as gpd

gdf = gpd.GeoDataFrame(
    data_geo_,
    geometry=gpd.points_from_xy(data_geo_["lng"], data_geo_["lat"])
)

# Load a world map for plotting
world = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))
# us = world[world['name'] == "United States"]

# Plot the map in the background
fig, ax = plt.subplots(figsize=(8, 6))
world.plot(ax=ax, color="white")

# ?Normalize the hours for color mapping
norm = plt.Normalize(
    vmin=gdf["total_hours_sum"].min(),
    vmax=gdf["total_hours_sum"].max()
)
cmap = plt.cm.Jet

# Plot the cities on top of the US map, color and size by total_hours_sum
gdf.plot(
    ax=ax,
    # ?
    color=gdf["total_hours_sum"].apply(lambda x: cmap(norm(x))),
    markersize=gdf["total_hours_sum"] / 500000,
    alpha=0.6
)

# Add labels to cities with total_hours_sum > 10,000
for x, y, label, hours in zip(
    gdf.geometry.x, gdf.geometry.y, gdf["city"], gdf["total_hours_sum"]
):
    if hours > 20000000:
        ax.text(x, y, label, fontsize=8, ha="right")

# Add a color bar to show the scale of total_hours_sum
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])  # Only needed for colorbar
cbar = plt.colorbar(sm, ax=ax, shrink=0.5)
# cbar.set_label("Total Hours")

ax.set_axis_off()

# plt.title("US Cities with Total Nurse Hours")
plt.xlim([-130, -65])  # Focus on the US longitude range
plt.ylim([20, 50])     # Focus on the US latitude range
plt.show()

### Join seasonal data

## Analyze geography

## Analyze seasonality

## Model

## Extra visualizations

### Sparklines

In [22]:
# Plot sparklines of average work hours across 91 days by state
(
    GT(gt_df.head(), rowname_col="STATE")
    .fmt_nanoplot(
        columns="lines",
        reference_line="mean",
        reference_area=["min", "q1"]
    )
    .tab_header(
        title="Nurse hours worked in the United States",
        subtitle="The top 5 busiest states",
    )
    .tab_stubhead(label="State")
    .cols_label(
        lines="Total hours worked over 91 days",
    )
)

## Concluding thoughts

(see <a href="#nte-rec1" class="quarto-xref">Note 1</a>)